In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive"

 2111B747-B05F-46FC-B033-A2AE18F50DAB.MOV
'BAO CAO NCKH.pdf'
'Bộ chấp hành điện 2'
'Chapter 04-Supply and Demand.pdf'
'Chính trị-xã hội'
 Classroom
'Cơ học'
'Colab Notebooks'
 ĐAMH
'Điện-Điện tử'
 Duy
 Fundamentals-of-Modern-Manufacturing-4th-Edition-by-Mikell-P-Groover.pdf
 IMG_0190.JPG
'KEYWIN(BANQUYEN).txt'
'Phan tu dan huong 2'
 sachcu-dangban
'Untitled Diagram.drawio'


In [0]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from matplotlib import rc
import matplotlib.ticker as ticker
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from google.colab import files
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import preprocessing 
%matplotlib inline

In [0]:
pd.options.display.float_format = '{:.5f}'.format

In [0]:
data=pd.read_csv("/content/drive/My Drive/Colab Notebooks/2008.csv")

In [6]:
data.shape

(7009728, 29)

In [0]:
pd.set_option('display.max_columns', 500,'display.max_rows', 500)

In [8]:
data.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance               int64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
CancellationCode      object
Diverted               int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtype: object

In [9]:
(data.isnull().sum()/len(data))*100

Year                 0.00000
Month                0.00000
DayofMonth           0.00000
DayOfWeek            0.00000
DepTime              1.94367
CRSDepTime           0.00000
ArrTime              2.16341
CRSArrTime           0.00000
UniqueCarrier        0.00000
FlightNum            0.00000
TailNum              1.18928
ActualElapsedTime    2.20692
CRSElapsedTime       0.01204
AirTime              2.20692
ArrDelay             2.20692
DepDelay             1.94367
Origin               0.00000
Dest                 0.00000
Distance             0.00000
TaxiIn               2.16341
TaxiOut              1.95525
Cancelled            0.00000
CancellationCode    98.03938
Diverted             0.00000
CarrierDelay        78.24830
WeatherDelay        78.24830
NASDelay            78.24830
SecurityDelay       78.24830
LateAircraftDelay   78.24830
dtype: float64

In [0]:
data=data.drop(columns=['CancellationCode','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','Year','Cancelled','Diverted',])

In [11]:
obj=data.select_dtypes(['object']).columns

for i in obj:
    print( str(i) + ':'+ str(len(data[i].value_counts())))

UniqueCarrier:20
TailNum:5373
Origin:303
Dest:304


In [0]:
data=data.drop(columns=['TailNum'])

In [0]:
data=data.drop(columns=['DepTime','ArrTime'])

In [0]:
data=data.dropna()

In [15]:
data.isnull().sum()

Month                0
DayofMonth           0
DayOfWeek            0
CRSDepTime           0
CRSArrTime           0
UniqueCarrier        0
FlightNum            0
ActualElapsedTime    0
CRSElapsedTime       0
AirTime              0
ArrDelay             0
DepDelay             0
Origin               0
Dest                 0
Distance             0
TaxiIn               0
TaxiOut              0
dtype: int64

In [16]:
data.shape

(6855029, 17)

In [17]:
data.head()

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,3,4,1955,2225,WN,335,128.00000,150.00000,116.00000,-14.00000,8.00000,IAD,TPA,810,4.00000,8.00000
1,1,3,4,735,1000,WN,3231,128.00000,145.00000,113.00000,2.00000,19.00000,IAD,TPA,810,5.00000,10.00000
2,1,3,4,620,750,WN,448,96.00000,90.00000,76.00000,14.00000,8.00000,IND,BWI,515,3.00000,17.00000
3,1,3,4,930,1100,WN,1746,88.00000,90.00000,78.00000,-6.00000,-4.00000,IND,BWI,515,3.00000,7.00000
4,1,3,4,1755,1925,WN,3920,90.00000,90.00000,77.00000,34.00000,34.00000,IND,BWI,515,3.00000,10.00000


In [18]:
data.describe(include="all")

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
count,6855029.00000,6855029.00000,6855029.00000,6855029.00000,6855029.00000,6855029,6855029.00000,6855029.00000,6855029.00000,6855029.00000,6855029.00000,6855029.00000,6855029,6855029,6855029.00000,6855029.00000,6855029.00000
unique,nan,nan,nan,nan,nan,20,nan,nan,nan,nan,nan,nan,303,302,nan,nan,nan
top,nan,nan,nan,nan,nan,WN,nan,nan,nan,nan,nan,nan,ATL,ATL,nan,nan,nan
freq,nan,nan,nan,nan,nan,1186911,nan,nan,nan,nan,nan,nan,407701,406744,nan,nan,nan
mean,6.38852,15.73826,3.92580,1325.03728,1493.71471,NaN,2212.50729,127.32242,129.07781,104.01859,8.16845,9.92384,NaN,NaN,728.74383,6.86015,16.44368
std,3.39883,8.80109,1.98846,464.31295,482.66419,NaN,1956.43672,70.18731,69.53271,67.43980,38.50194,35.20386,NaN,NaN,563.24473,4.93122,11.31062
min,1.00000,1.00000,1.00000,0.00000,0.00000,NaN,1.00000,12.00000,-141.00000,0.00000,-519.00000,-534.00000,NaN,NaN,11.00000,0.00000,0.00000
25%,3.00000,8.00000,2.00000,925.00000,1115.00000,NaN,618.00000,77.00000,80.00000,55.00000,-10.00000,-4.00000,NaN,NaN,326.00000,4.00000,10.00000
50%,6.00000,16.00000,4.00000,1317.00000,1515.00000,NaN,1563.00000,110.00000,111.00000,86.00000,-2.00000,-1.00000,NaN,NaN,581.00000,6.00000,14.00000
75%,9.00000,23.00000,6.00000,1715.00000,1905.00000,NaN,3494.00000,157.00000,159.00000,132.00000,12.00000,8.00000,NaN,NaN,954.00000,8.00000,19.00000


In [0]:
#Delete 5.800.000 rows.
data=data.reset_index()
data=data.drop(columns=['index'])
a=np.arange(5800000)
data=data.drop(a)
data=data.reset_index()
data=data.drop(columns=['index'])

In [0]:
data1 = data

Change CRSArrTime and CRSDepTime to hour categories.

In [0]:
def change_time(x):
  min=x%100;
  hour=(x-min)/100
  res = hour + round(min/60)
  if (res == 24):
    res=0
  return res

In [0]:
data1['CRSDepTime']=data1['CRSDepTime'].apply(change_time)

In [23]:
data1['CRSDepTime']

0         17.00000
1         12.00000
2         16.00000
3         16.00000
4         19.00000
            ...   
1055024   10.00000
1055025    9.00000
1055026    7.00000
1055027   13.00000
1055028   11.00000
Name: CRSDepTime, Length: 1055029, dtype: float64

In [0]:
data1['CRSArrTime']=data1['CRSArrTime'].apply(change_time)

In [25]:
data1['CRSArrTime']

0         18.00000
1         15.00000
2         18.00000
3         18.00000
4         20.00000
            ...   
1055024   12.00000
1055025   10.00000
1055026    9.00000
1055027   15.00000
1055028   14.00000
Name: CRSArrTime, Length: 1055029, dtype: float64

In [26]:
data1.describe(include="all")

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
count,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029,1055029,1055029.00000,1055029.00000,1055029.00000
unique,nan,nan,nan,nan,nan,19,nan,nan,nan,nan,nan,nan,283,283,nan,nan,nan
top,nan,nan,nan,nan,nan,WN,nan,nan,nan,nan,nan,nan,ATL,ATL,nan,nan,nan
freq,nan,nan,nan,nan,nan,190225,nan,nan,nan,nan,nan,nan,71092,71033,nan,nan,nan
mean,11.48571,15.53451,3.91031,13.28685,14.83719,NaN,2242.38298,127.96587,129.86000,104.23996,9.34127,11.23540,NaN,NaN,724.89701,7.07758,16.64833
std,0.52262,8.76210,2.03837,4.60422,4.92641,NaN,1967.09954,69.78763,69.48844,67.27730,41.52964,37.67829,NaN,NaN,555.23860,5.44455,11.21758
min,10.00000,1.00000,1.00000,0.00000,0.00000,NaN,1.00000,12.00000,16.00000,6.00000,-90.00000,-49.00000,NaN,NaN,24.00000,0.00000,1.00000
25%,11.00000,8.00000,2.00000,9.00000,11.00000,NaN,670.00000,78.00000,80.00000,56.00000,-11.00000,-4.00000,NaN,NaN,329.00000,4.00000,10.00000
50%,11.00000,15.00000,4.00000,13.00000,15.00000,NaN,1608.00000,111.00000,112.00000,86.00000,-2.00000,-1.00000,NaN,NaN,581.00000,6.00000,14.00000
75%,12.00000,23.00000,6.00000,17.00000,19.00000,NaN,3119.00000,158.00000,160.00000,132.00000,13.00000,9.00000,NaN,NaN,948.00000,8.00000,19.00000


1 Hot encoding and Min max scaler

In [27]:
obj=['UniqueCarrier', 'Origin', 'Dest','Month','DayofMonth','DayOfWeek','CRSDepTime','CRSArrTime','FlightNum']

for i in obj:
    print( str(i) + ':'+ str(len(data1[i].value_counts())))

UniqueCarrier:19
Origin:283
Dest:283
Month:3
DayofMonth:31
DayOfWeek:7
CRSDepTime:24
CRSArrTime:24
FlightNum:7104


FlightNum > 500. Delete FlightNum.

In [0]:
data1=data1.drop(columns=['FlightNum'])

In [29]:
data1.select_dtypes(['object']).columns

Index(['UniqueCarrier', 'Origin', 'Dest'], dtype='object')

In [0]:
data_1hot=data1

In [0]:
obj=['UniqueCarrier', 'Origin', 'Dest', 'Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime']

for i in obj:
    data_1hot = pd.get_dummies(data_1hot, columns=[i])

In [32]:
data_1hot.describe(include="all")

,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,UniqueCarrier_9E,UniqueCarrier_AA,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV,Origin_ABE,Origin_ABI,Origin_ABQ,Origin_ABY,Origin_ACT,Origin_ACV,Origin_ADK,Origin_ADQ,Origin_AEX,Origin_AGS,Origin_ALB,Origin_ALO,Origin_AMA,Origin_ANC,Origin_ASE,Origin_ATL,Origin_ATW,Origin_AUS,Origin_AVL,Origin_AVP,Origin_AZO,Origin_BDL,Origin_BET,Origin_BFL,Origin_BGM,Origin_BGR,Origin_BHM,Origin_BIL,Origin_BIS,Origin_BMI,Origin_BNA,Origin_BOI,Origin_BOS,Origin_BQK,Origin_BQN,Origin_BRO,Origin_BRW,Origin_BTM,Origin_BTR,Origin_BTV,Origin_BUF,Origin_BUR,Origin_BWI,Origin_BZN,Origin_CAE,Origin_CAK,Origin_CDC,Origin_CDV,Origin_CEC,Origin_CHA,Origin_CHO,Origin_CHS,Origin_CIC,Origin_CID,Origin_CLD,Origin_CLE,Origin_CLL,Origin_CLT,Origin_CMH,Origin_CMI,Origin_CMX,Origin_COD,Origin_COS,Origin_CPR,Origin_CRP,Origin_CRW,Origin_CSG,Origin_CVG,Origin_CWA,Origin_DAB,Origin_DAL,Origin_DAY,Origin_DBQ,Origin_DCA,Origin_DEN,Origin_DFW,Origin_DHN,Origin_DLH,Origin_DRO,Origin_DSM,Origin_DTW,Origin_EGE,Origin_EKO,Origin_ELM,Origin_ELP,Origin_ERI,Origin_EUG,Origin_EVV,Origin_EWN,Origin_EWR,Origin_EYW,Origin_FAI,Origin_FAR,Origin_FAT,Origin_FAY,Origin_FCA,Origin_FLG,Origin_FLL,Origin_FLO,Origin_FNT,Origin_FSD,Origin_FSM,Origin_FWA,Origin_GCC,Origin_GEG,Origin_GFK,Origin_GGG,Origin_GJT,Origin_GNV,Origin_GPT,Origin_GRB,Origin_GRK,Origin_GRR,Origin_GSO,Origin_GSP,Origin_GTF,Origin_GTR,Origin_GUC,Origin_HDN,Origin_HHH,Origin_HLN,Origin_HNL,Origin_HOU,Origin_HPN,Origin_HRL,Origin_HSV,Origin_HTS,Origin_IAD,Origin_IAH,Origin_ICT,Origin_IDA,Origin_ILM,Origin_IND,Origin_IPL,Origin_ISP,Origin_ITH,Origin_ITO,Origin_IYK,Origin_JAC,Origin_JAN,Origin_JAX,Origin_JFK,Origin_JNU,Origin_KOA,Origin_KTN,Origin_LAN,Origin_LAS,Origin_LAW,Origin_LAX,Origin_LBB,Origin_LCH,Origin_LEX,Origin_LFT,Origin_LGA,Origin_LGB,Origin_LIH,Origin_LIT,Origin_LMT,Origin_LNK,Origin_LRD,Origin_LSE,Origin_LWB,Origin_LWS,Origin_LYH,Origin_MAF,Origin_MBS,Origin_MCI,Origin_MCO,Origin_MDT,Origin_MDW,Origin_MEI,Origin_MEM,Origin_MFE,Origin_MFR,Origin_MGM,Origin_MHT,Origin_MIA,Origin_MKE,Origin_MLB,Origin_MLI,Origin_MLU,Origin_MOB,Origin_MOD,Origin_MOT,Origin_MQT,Origin_MRY,Origin_MSN,Origin_MSO,Origin_MSP,Origin_MSY,Origin_MTJ,Origin_MYR,Origin_OAJ,Origin_OAK,Origin_OGG,Origin_OKC,Origin_OMA,Origin_OME,Origin_ONT,Origin_ORD,Origin_ORF,Origin_OTH,Origin_OTZ,Origin_OXR,Origin_PBI,Origin_PDX,Origin_PFN,Origin_PHF,Origin_PHL,Origin_PHX,Origin_PIA,Origin_PIH,Origin_PIR,Origin_PIT,Origin_PMD,Origin_PNS,Origin_PSC,Origin_PSE,Origin_PSG,Origin_PSP,Origin_PVD,Origin_PWM,Origin_RAP,...,Dest_HOU,Dest_HPN,Dest_HRL,Dest_HSV,Dest_HTS,Dest_IAD,Dest_IAH,Dest_ICT,Dest_IDA,Dest_ILM,Dest_IND,Dest_IPL,Dest_ISP,Dest_ITH,Dest_ITO,Dest_IYK,Dest_JAC,Dest_JAN,Dest_JAX,Dest_JFK,Dest_JNU,Dest_KOA,Dest_KTN,Dest_LAN,Dest_LAS,Dest_LAW,Dest_LAX,Dest_LBB,Dest_LCH,Dest_LEX,Dest_LFT,Dest_LGA,Dest_LGB,Dest_LIH,Dest_LIT,Dest_LMT,Dest_LNK,Dest_LRD,Dest_LSE,Dest_LWB,Dest_LWS,Dest_LYH,Dest_MAF,Dest_MBS,Dest_MCI,Dest_MCO,Dest_MDT,Dest_MDW,Dest_MEI,Dest_MEM,Dest_MFE,Dest_MFR,Dest_MGM,Dest_MHT,Dest_MIA,Dest_MKE,Dest_MLB,Dest_MLI,Dest_MLU,Dest_MOB,Dest_MOD,Dest_MOT,Dest_MQT,Dest_MRY,Dest_MSN,Dest_MSO,Dest_MSP,Dest_MSY,Dest_MTJ,Dest_MYR,Dest_OAJ,Dest_OAK,Dest_OGG,Dest_OKC,Dest_OMA,Dest_OME,Dest_ONT,Dest_ORD,Dest_ORF,Dest_OTH,Dest_OTZ,Dest_OXR,Dest_PBI,Dest_PDX,Dest_PFN,Dest_PHF,Dest_PHL,Dest_PHX,Dest_PIA,Dest_PIH,Dest_PIR,Dest_PIT,Dest_PMD,Dest_PNS,Dest_PSC,Dest_PSE,Dest_PSG,Dest_PSP,Dest_PVD,Dest_PWM,Dest_RAP,Dest_RDD,Dest_RDM,Dest_RDU,Dest_RHI,Dest_RIC,Dest_RKS,Dest_RNO,Dest_ROA,Dest_ROC,Dest_ROW,Dest_RST,Dest_RSW,Dest_SAN,Dest_SAT,Dest_SAV,Dest_SBA,Dest_SBN,Dest_SBP,Dest_SCC,Dest_SCE,Dest_SDF,Dest_SEA,Dest_SFO,Dest_SGF,Dest_SGU,De

Apply MinMaxScaler

In [0]:
#scaler = MinMaxScaler()
#print(scaler.fit(data))
#data1=scaler.transform(data)
#data2=pd.DataFrame(data1, columns=data.columns)

In [34]:
data1.describe(include="all")

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
count,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029.00000,1055029,1055029,1055029.00000,1055029.00000,1055029.00000
unique,nan,nan,nan,nan,nan,19,nan,nan,nan,nan,nan,283,283,nan,nan,nan
top,nan,nan,nan,nan,nan,WN,nan,nan,nan,nan,nan,ATL,ATL,nan,nan,nan
freq,nan,nan,nan,nan,nan,190225,nan,nan,nan,nan,nan,71092,71033,nan,nan,nan
mean,11.48571,15.53451,3.91031,13.28685,14.83719,NaN,127.96587,129.86000,104.23996,9.34127,11.23540,NaN,NaN,724.89701,7.07758,16.64833
std,0.52262,8.76210,2.03837,4.60422,4.92641,NaN,69.78763,69.48844,67.27730,41.52964,37.67829,NaN,NaN,555.23860,5.44455,11.21758
min,10.00000,1.00000,1.00000,0.00000,0.00000,NaN,12.00000,16.00000,6.00000,-90.00000,-49.00000,NaN,NaN,24.00000,0.00000,1.00000
25%,11.00000,8.00000,2.00000,9.00000,11.00000,NaN,78.00000,80.00000,56.00000,-11.00000,-4.00000,NaN,NaN,329.00000,4.00000,10.00000
50%,11.00000,15.00000,4.00000,13.00000,15.00000,NaN,111.00000,112.00000,86.00000,-2.00000,-1.00000,NaN,NaN,581.00000,6.00000,14.00000
75%,12.00000,23.00000,6.00000,17.00000,19.00000,NaN,158.00000,160.00000,132.00000,13.00000,9.00000,NaN,NaN,948.00000,8.00000,19.00000


**Apply Naïve Bayes Model**

Prepare data_train and data_test.

In [0]:
data_NB_target = pd.DataFrame({'ArrDelay':data_1hot['ArrDelay'], 'DepDelay': data_1hot['DepDelay'], 'Delay':np.arange( len(data_1hot['AirTime']) )})

In [36]:
data_NB_target.head()

,ArrDelay,DepDelay,Delay
0,-3.00000,-1.00000,0
1,-13.00000,0.00000,1
2,40.00000,38.00000,2
3,-5.00000,2.00000,3
4,16.00000,10.00000,4


In [0]:
data_NB_target = data_NB_target.astype({"ArrDelay":'int64', "DepDelay":'int64', "Delay":'int64'})

In [38]:
data_NB_target.dtypes

ArrDelay    int64
DepDelay    int64
Delay       int64
dtype: object

In [0]:
def consider_delay(x,y):
  if ((x >= 30)or(y >= 30)):
    return 1
  return 0

In [0]:
data_NB_target['Delay'] = data_NB_target.apply(lambda x: consider_delay(x.ArrDelay, x.DepDelay), axis=1)

In [0]:
data_NB_target=data_NB_target.drop(columns=['ArrDelay','DepDelay'])

In [0]:
data_NB_train=data_1hot.drop(columns=['ArrDelay','DepDelay'])

In [43]:
data_NB_target.head()

,Delay
0,0
1,0
2,1
3,0
4,0


In [44]:
data_NB_train.head()

,ActualElapsedTime,CRSElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,UniqueCarrier_9E,UniqueCarrier_AA,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV,Origin_ABE,Origin_ABI,Origin_ABQ,Origin_ABY,Origin_ACT,Origin_ACV,Origin_ADK,Origin_ADQ,Origin_AEX,Origin_AGS,Origin_ALB,Origin_ALO,Origin_AMA,Origin_ANC,Origin_ASE,Origin_ATL,Origin_ATW,Origin_AUS,Origin_AVL,Origin_AVP,Origin_AZO,Origin_BDL,Origin_BET,Origin_BFL,Origin_BGM,Origin_BGR,Origin_BHM,Origin_BIL,Origin_BIS,Origin_BMI,Origin_BNA,Origin_BOI,Origin_BOS,Origin_BQK,Origin_BQN,Origin_BRO,Origin_BRW,Origin_BTM,Origin_BTR,Origin_BTV,Origin_BUF,Origin_BUR,Origin_BWI,Origin_BZN,Origin_CAE,Origin_CAK,Origin_CDC,Origin_CDV,Origin_CEC,Origin_CHA,Origin_CHO,Origin_CHS,Origin_CIC,Origin_CID,Origin_CLD,Origin_CLE,Origin_CLL,Origin_CLT,Origin_CMH,Origin_CMI,Origin_CMX,Origin_COD,Origin_COS,Origin_CPR,Origin_CRP,Origin_CRW,Origin_CSG,Origin_CVG,Origin_CWA,Origin_DAB,Origin_DAL,Origin_DAY,Origin_DBQ,Origin_DCA,Origin_DEN,Origin_DFW,Origin_DHN,Origin_DLH,Origin_DRO,Origin_DSM,Origin_DTW,Origin_EGE,Origin_EKO,Origin_ELM,Origin_ELP,Origin_ERI,Origin_EUG,Origin_EVV,Origin_EWN,Origin_EWR,Origin_EYW,Origin_FAI,Origin_FAR,Origin_FAT,Origin_FAY,Origin_FCA,Origin_FLG,Origin_FLL,Origin_FLO,Origin_FNT,Origin_FSD,Origin_FSM,Origin_FWA,Origin_GCC,Origin_GEG,Origin_GFK,Origin_GGG,Origin_GJT,Origin_GNV,Origin_GPT,Origin_GRB,Origin_GRK,Origin_GRR,Origin_GSO,Origin_GSP,Origin_GTF,Origin_GTR,Origin_GUC,Origin_HDN,Origin_HHH,Origin_HLN,Origin_HNL,Origin_HOU,Origin_HPN,Origin_HRL,Origin_HSV,Origin_HTS,Origin_IAD,Origin_IAH,Origin_ICT,Origin_IDA,Origin_ILM,Origin_IND,Origin_IPL,Origin_ISP,Origin_ITH,Origin_ITO,Origin_IYK,Origin_JAC,Origin_JAN,Origin_JAX,Origin_JFK,Origin_JNU,Origin_KOA,Origin_KTN,Origin_LAN,Origin_LAS,Origin_LAW,Origin_LAX,Origin_LBB,Origin_LCH,Origin_LEX,Origin_LFT,Origin_LGA,Origin_LGB,Origin_LIH,Origin_LIT,Origin_LMT,Origin_LNK,Origin_LRD,Origin_LSE,Origin_LWB,Origin_LWS,Origin_LYH,Origin_MAF,Origin_MBS,Origin_MCI,Origin_MCO,Origin_MDT,Origin_MDW,Origin_MEI,Origin_MEM,Origin_MFE,Origin_MFR,Origin_MGM,Origin_MHT,Origin_MIA,Origin_MKE,Origin_MLB,Origin_MLI,Origin_MLU,Origin_MOB,Origin_MOD,Origin_MOT,Origin_MQT,Origin_MRY,Origin_MSN,Origin_MSO,Origin_MSP,Origin_MSY,Origin_MTJ,Origin_MYR,Origin_OAJ,Origin_OAK,Origin_OGG,Origin_OKC,Origin_OMA,Origin_OME,Origin_ONT,Origin_ORD,Origin_ORF,Origin_OTH,Origin_OTZ,Origin_OXR,Origin_PBI,Origin_PDX,Origin_PFN,Origin_PHF,Origin_PHL,Origin_PHX,Origin_PIA,Origin_PIH,Origin_PIR,Origin_PIT,Origin_PMD,Origin_PNS,Origin_PSC,Origin_PSE,Origin_PSG,Origin_PSP,Origin_PVD,Origin_PWM,Origin_RAP,Origin_RDD,Origin_RDM,...,Dest_HOU,Dest_HPN,Dest_HRL,Dest_HSV,Dest_HTS,Dest_IAD,Dest_IAH,Dest_ICT,Dest_IDA,Dest_ILM,Dest_IND,Dest_IPL,Dest_ISP,Dest_ITH,Dest_ITO,Dest_IYK,Dest_JAC,Dest_JAN,Dest_JAX,Dest_JFK,Dest_JNU,Dest_KOA,Dest_KTN,Dest_LAN,Dest_LAS,Dest_LAW,Dest_LAX,Dest_LBB,Dest_LCH,Dest_LEX,Dest_LFT,Dest_LGA,Dest_LGB,Dest_LIH,Dest_LIT,Dest_LMT,Dest_LNK,Dest_LRD,Dest_LSE,Dest_LWB,Dest_LWS,Dest_LYH,Dest_MAF,Dest_MBS,Dest_MCI,Dest_MCO,Dest_MDT,Dest_MDW,Dest_MEI,Dest_MEM,Dest_MFE,Dest_MFR,Dest_MGM,Dest_MHT,Dest_MIA,Dest_MKE,Dest_MLB,Dest_MLI,Dest_MLU,Dest_MOB,Dest_MOD,Dest_MOT,Dest_MQT,Dest_MRY,Dest_MSN,Dest_MSO,Dest_MSP,Dest_MSY,Dest_MTJ,Dest_MYR,Dest_OAJ,Dest_OAK,Dest_OGG,Dest_OKC,Dest_OMA,Dest_OME,Dest_ONT,Dest_ORD,Dest_ORF,Dest_OTH,Dest_OTZ,Dest_OXR,Dest_PBI,Dest_PDX,Dest_PFN,Dest_PHF,Dest_PHL,Dest_PHX,Dest_PIA,Dest_PIH,Dest_PIR,Dest_PIT,Dest_PMD,Dest_PNS,Dest_PSC,Dest_PSE,Dest_PSG,Dest_PSP,Dest_PVD,Dest_PWM,Dest_RAP,Dest_RDD,Dest_RDM,Dest_RDU,Dest_RHI,Dest_RIC,Dest_RKS,Dest_RNO,Dest_ROA,Dest_ROC,Dest_ROW,Dest_RST,Dest_RSW,Dest_SAN,Dest_SAT,Dest_SAV,Dest_SBA,Dest_SBN,Dest_SBP,Dest_SCC,Dest_SCE,Dest_SDF,Dest_SEA,Dest_SFO,Dest_SGF,Dest_SG

Apply Minmaxscaler

In [0]:
scaler = MinMaxScaler(copy=True, feature_range=(0.1, 1))

In [46]:
print(scaler.fit(data_NB_train))

MinMaxScaler(copy=True, feature_range=(0.1, 1))


In [0]:
data_NB_train_mmc=scaler.transform(data_NB_train)


In [0]:
data_NB_train_mmc=pd.DataFrame(data_NB_train_mmc, columns=data_NB_train.columns)

Hold out Validation.

In [0]:
data_train, data_test, data_train_target, data_test_target = train_test_split(
...     data_NB_train, data_NB_target, test_size=0.2, random_state=11)

In [50]:
clf_pf = GaussianNB()
clf_pf.partial_fit(data_train, data_train_target, np.unique(data_train_target))
print(clf_pf.score)

NB_predictions_train=clf_pf.predict(data_train)
NB_predictions_test=clf_pf.predict(data_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<bound method ClassifierMixin.score of GaussianNB(priors=None, var_smoothing=1e-09)>


In [51]:
print('Train RMSE:', np.sqrt(mean_squared_error(data_train_target,NB_predictions_train)))
print('Test RMSE :', np.sqrt(mean_squared_error(data_test_target, NB_predictions_test)))

Train RMSE: 0.6875631180936168
Test RMSE : 0.688535624165417


In [61]:
print("Accuracy:",metrics.accuracy_score(data_train_target, NB_predictions_train))
print("Accuracy:",metrics.accuracy_score(data_test_target, NB_predictions_test))

Accuracy: 0.5272569586373831
Accuracy: 0.5259186942551397


In [63]:
data1

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,10,5,7,17.00000,18.00000,DL,128.00000,130.00000,101.00000,-3.00000,-1.00000,ATL,ORD,606,7.00000,20.00000
1,10,5,7,12.00000,15.00000,DL,106.00000,119.00000,86.00000,-13.00000,0.00000,ORD,ATL,606,8.00000,12.00000
2,10,5,7,16.00000,18.00000,DL,106.00000,104.00000,78.00000,40.00000,38.00000,ATL,PBI,545,6.00000,22.00000
3,10,5,7,16.00000,18.00000,DL,63.00000,70.00000,46.00000,-5.00000,2.00000,MOB,ATL,302,6.00000,11.00000
4,10,5,7,19.00000,20.00000,DL,134.00000,128.00000,96.00000,16.00000,10.00000,ATL,ORD,606,10.00000,28.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055024,12,13,6,10.00000,12.00000,DL,122.00000,111.00000,71.00000,14.00000,3.00000,ATL,IAD,533,6.00000,45.00000
1055025,12,13,6,9.00000,10.00000,DL,167.00000,168.00000,139.00000,-2.00000,-1.00000,ATL,SAT,874,5.00000,23.00000
1055026,12,13,6,7.00000,9.00000,DL,121.00000,116.00000,85.00000,0.00000,-5.00000,PBI,ATL,545,24.00000,12.00000
1055027,12,13,6,13.00000,15.00000,DL,115.00000,117.00000,89.00000,9.00000,11.00000,IAD,ATL,533,13.00000,13.00000


In [0]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
data_label_enc=data1
# Encode labels in column 'species'. 

obj=data.select_dtypes(['object']).columns

for i in obj:
    data_label_enc[i]= label_encoder.fit_transform(data1[i]) 
  

In [68]:
data_label_enc.drop(columns=['ArrDelay','DepDelay'])

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,ActualElapsedTime,CRSElapsedTime,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut
0,10,5,7,17.00000,18.00000,5,128.00000,130.00000,101.00000,15,199,606,7.00000,20.00000
1,10,5,7,12.00000,15.00000,5,106.00000,119.00000,86.00000,199,15,606,8.00000,12.00000
2,10,5,7,16.00000,18.00000,5,106.00000,104.00000,78.00000,15,204,545,6.00000,22.00000
3,10,5,7,16.00000,18.00000,5,63.00000,70.00000,46.00000,181,15,302,6.00000,11.00000
4,10,5,7,19.00000,20.00000,5,134.00000,128.00000,96.00000,15,199,606,10.00000,28.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055024,12,13,6,10.00000,12.00000,5,122.00000,111.00000,71.00000,15,127,533,6.00000,45.00000
1055025,12,13,6,9.00000,10.00000,5,167.00000,168.00000,139.00000,15,236,874,5.00000,23.00000
1055026,12,13,6,7.00000,9.00000,5,121.00000,116.00000,85.00000,204,15,545,24.00000,12.00000
1055027,12,13,6,13.00000,15.00000,5,115.00000,117.00000,89.00000,127,15,533,13.00000,13.00000


Hold out Validation.

In [0]:
data_train, data_test, data_train_target, data_test_target = train_test_split(
...     data_label_enc, data_NB_target, test_size=0.2, random_state=12)

In [71]:
clf_pf = GaussianNB()
clf_pf.partial_fit(data_train, data_train_target, np.unique(data_train_target))
print(clf_pf.score)

NB_predictions_train=clf_pf.predict(data_train)
NB_predictions_test=clf_pf.predict(data_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<bound method ClassifierMixin.score of GaussianNB(priors=None, var_smoothing=1e-09)>


In [72]:
print("Accuracy:",metrics.accuracy_score(data_train_target, NB_predictions_train))
print("Accuracy:",metrics.accuracy_score(data_test_target, NB_predictions_test))

Accuracy: 0.9669760184260382
Accuracy: 0.9668729799152631
